<a href="https://colab.research.google.com/github/ferfsnandes/TCC--usando-Machine-Leraning-para-prever-o-risco-de-doencas-cronicas-nao-transmissiveis-em-idosos/blob/main/TCC_Gabriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://www.geeksforgeeks.org/machine-learning/?ref=lbp

https://www.geeksforgeeks.org/working-with-missing-data-in-pandas/


In [ ]:
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/database_arrumada (2).csv")


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Columns: 330 entries, Bancos to escola_cat
dtypes: float64(300), int64(9), object(21)
memory usage: 2.0+ MB


In [ ]:
empty_cols = [col for col in df.columns if df[col].isna().all() or df[col].isnull().all()]
len(empty_cols)
print(len(empty_cols))
print(f"Colunas Vazias: {empty_cols}")

0
Colunas Vazias: []


In [ ]:
df.drop(empty_cols, axis=1, inplace=True)


ToDO: Threshold 90% = drop
ToDO: Threshold 75% = drop (if doesnt have clear impact on the model)
ToDO: drop colluns without any impact in the model



In [ ]:
threshold = 0.9
empty_cols = [col for col in df.columns if df[col].isnull().sum() / len(df) > threshold]
print(f"{len(empty_cols)}")
print(f"{empty_cols}")


2
['H10_quantos_dentes', 'R3_relig_espiritual_sem_religiao']


R3= Tem alguma religiosidade, espiritualidade sem religião?



In [ ]:
df.drop(columns=['R3_relig_espiritual_sem_religiao'], inplace=True)
df['H10_quantos_dentes'].fillna(0, inplace=True)


In [ ]:
threshold = 0.75
empty_cols = [col for col in df.columns if df[col].isnull().sum() / len(df) > threshold and df[col].isnull().sum() / len(df) < 0.9]
print(f"{len(empty_cols)}")
print(f"{empty_cols}")

24
['C48_CDR_pontos_Morris', 'F9_dias_ciclismo', 'F10_minutos_ciclismo', 'F13_dias_danca_salao', 'F14_minutos_danca_salao', 'F21_dias_ginastica_fora_casa', 'F22_minutos_ginastica_fora_casa', 'F25_dias_hidroginastica', 'F26_minutos_hidroginastica', 'F29_dias_corrida_leve_caminh_vigorosa', 'F30_minutos_corrida_leve_caminh_vigorosa', 'F33_dias_corrida_vigorosa', 'F34_minutos_corrida_vigorosa', 'F37_dias_musculacao', 'F38_minutos_musculacao', 'F41_dias_natacao_academia_campo_aberto', 'F42_minutos_natacao_academia_campo_aberto', 'F45_dias_voleibol', 'F46_minutos_voleibol', 'J45_manter_se_atualizado', 'J46_capaz_prestar_atencao_entender_programas', 'R4_import_relig_espiritual_vida', 'CPF_idosos_Brasilia', 'N_protocolo_Idosos_Brasilia']


ToDo: if ( Threshold > 75% and Threshold < 90% and has clear impact ) = analise to fill


In [ ]:
df.drop(columns=['CPF_idosos_Brasilia', 'N_protocolo_Idosos_Brasilia'], inplace=True)


In [ ]:
df['C48_CDR_pontos_Morris'].fillna(0, inplace=True)

ToDo: if ( Threshold > 50% and Threshold < 75 % and has clear impact ) = analise to fill

In [ ]:


threshold = 0.5
empty_cols = [col for col in df.columns if df[col].isnull().sum() / len(df) > threshold and df[col].isnull().sum() / len(df) < 0.75]
print(f"{len(empty_cols)}")
print(f"{empty_cols}")

101
['C41_memoria', 'C42_orientacao_temporal_espacial', 'C43_julgamento_solucao_problemas', 'C44_relacoes_comunitarias', 'C45_lar_passa_tempos', 'C46_cuidados_pessoais', 'C47_CDR_soma_pontos', 'C49_CDR_nivel_demencia_Morris', 'F2_quantos_quilos', 'F4_caminhada', 'F5_dias_caminhada', 'F6_minutos_caminhada', 'F7_caminhada_3.8_METS', 'F8_ciclismo', 'F11_ciclismo_4.0_METS', 'F12_danca_salao', 'F15_danca_salao_4.5_METS', 'F16_ginastica_casa', 'F17_dias_ginastica_casa', 'F18_minutos_ginastica_casa', 'F19_ginastica_casa_3.5_METS', 'F20_ginastica_fora_casa', 'F23_ginastica_fora_casa_6.0_METS', 'F24_hidroginastica', 'F27_hidroginastica_4.0_METS', 'F28_corrida_leve_caminh_vigorosa', 'F31_corrida_leve_6.0_METS', 'F32_corrida_vigorosa', 'F36_musculacao', 'F39_musculacao_3.0_METS', 'F40_natacao_academia_campo_aberto', 'F44_voleibol', 'F48_soma_METS', 'F50_fragilidade_atividade_fisica', 'H6_num_dentes_arcada_sup', 'H7_num_dentes_arcada_inf', 'H8_num_dentes_duas_arcadas', 'H16_boca_seca', 'J1_fazer_v

Verificando quais colunas são categóricas

In [ ]:
cols_with_only_1_2_null = [col for col in df.columns if df[col].isin([1, 2]).sum() + df[col].isnull().sum() == len(df)]
print(f"Numero de Colunas com Variaveis Categóricas: {len(cols_with_only_1_2_null)}")
print(f"Colunas Categóricas: {cols_with_only_1_2_null}")


Numero de Colunas com Variaveis Categóricas: 87
Colunas Categóricas: ['B3_sexo', 'B9_alfabetizado', 'B12_sozinho', 'B13_conjuge', 'B14_filhos', 'B15_netos', 'B16_bisnetos', 'B17_outro_parente', 'B23_proprietario_resid', 'F1_perda_peso', 'G1_coracao', 'G2_hipertensao', 'G3_AVC_isquemia', 'G4_diabetes', 'G5_cancer', 'G6_artrite_reumatismo', 'G7_doencas_pulmoes', 'G8_depressao', 'G9_osteoporose', 'G12_incontinencia_urinaria', 'G13_perda_apetite', 'G14_dificuldades_memoria', 'G15_lesao_pele_feridas', 'G16_dificuldade_engolir', 'G17_sensacao_alimento_parado', 'G18_retorno_alimento', 'G19_disfagia', 'G20_dor_cronica', 'G21_acorda_madrugada_nao_dorme', 'G22_acordado_maior_parte_noite', 'G23_demora_pegar_sono', 'G24_dorme_mal_noite', 'G26_insonia_classificacao', 'G27_cochilo_diurno', 'G28_consome_remedios', 'G30_polifarmacia', 'G31_medicamento_hipertensao', 'G32_insulina', 'G34_vitaminas', 'G35_medicamento_depressao', 'G36_fuma_atualmente', 'H1_comida_dura', 'H2_maca', 'H3_cenoura', 'H4_pao_to

Trocando 2 pelo 0 nas colunas Binarias para a facilitação de aprendizado do modelo


In [ ]:
for col in cols_with_only_1_2_null:
    df[col].replace({2: 0}, inplace=True)


In [ ]:
df['F8_ciclismo'].replace({2: 0}, inplace=True)
df['F12_danca_salao'].replace({2: 0}, inplace=True)

In [ ]:
df['F20_ginastica_fora_casa'].replace({2: 0}, inplace=True)
df['F24_hidroginastica'].replace({2: 0}, inplace=True)
df['F28_corrida_leve_caminh_vigorosa'].replace({2: 0}, inplace=True)
df['F32_corrida_vigorosa'].replace({2: 0}, inplace=True)
df['F36_musculacao'].replace({2: 0}, inplace=True)
df['F40_natacao_academia_campo_aberto'].replace({2: 0}, inplace=True)
df['F44_voleibol'].replace({2: 0}, inplace=True)

In [ ]:
threshold = 0.75
empty_cols = [col for col in df.columns if df[col].isnull().sum() / len(df) > threshold and df[col].isnull().sum() / len(df) < 0.9]
print(f"{len(empty_cols)}")
print(f"{empty_cols}")

21
['F9_dias_ciclismo', 'F10_minutos_ciclismo', 'F13_dias_danca_salao', 'F14_minutos_danca_salao', 'F21_dias_ginastica_fora_casa', 'F22_minutos_ginastica_fora_casa', 'F25_dias_hidroginastica', 'F26_minutos_hidroginastica', 'F29_dias_corrida_leve_caminh_vigorosa', 'F30_minutos_corrida_leve_caminh_vigorosa', 'F33_dias_corrida_vigorosa', 'F34_minutos_corrida_vigorosa', 'F37_dias_musculacao', 'F38_minutos_musculacao', 'F41_dias_natacao_academia_campo_aberto', 'F42_minutos_natacao_academia_campo_aberto', 'F45_dias_voleibol', 'F46_minutos_voleibol', 'J45_manter_se_atualizado', 'J46_capaz_prestar_atencao_entender_programas', 'R4_import_relig_espiritual_vida']


Tratamento dos dados : F8 = se a pessoa pratica ciclismo


F9 = quantos dias dias da semana ela pratica ciclismo
F10 = Quantos minutos cada praticá dura aproximadamente

ou seja se ela não pratica ambas as outras variaveis vão ser obrigatoriamente 0, por isso as que não estiverem preenchidas

a mesma situação  ocorrera semelhantemente para outras variaves semelhantes


In [ ]:
condition = (df['F8_ciclismo'] == 0) & (df['F9_dias_ciclismo'].isnull()) & (df['F10_minutos_ciclismo'].isnull())
df.loc[condition, ['F9_dias_ciclismo', 'F10_minutos_ciclismo']] = 0

In [ ]:
condition = (df['F12_danca_salao'] == 0) & (df['F13_dias_danca_salao'].isnull()) & (df['F14_minutos_danca_salao'].isnull())
df.loc[condition, ['F13_dias_danca_salao', 'F14_minutos_danca_salao']] = 0


In [ ]:
condition = (df['F20_ginastica_fora_casa'] == 0) & (df['F21_dias_ginastica_fora_casa'].isnull()) & (df['F22_minutos_ginastica_fora_casa'].isnull())
df.loc[condition, ['F21_dias_ginastica_fora_casa', 'F22_minutos_ginastica_fora_casa']] = 0

In [ ]:
condition = (df['F24_hidroginastica'] == 0) & (df['F25_dias_hidroginastica'].isnull()) & (df['F26_minutos_hidroginastica'].isnull())
df.loc[condition, ['F25_dias_hidroginastica', 'F26_minutos_hidroginastica']] = 0

In [ ]:
condition = (df['F28_corrida_leve_caminh_vigorosa'] == 0) & (df['F29_dias_corrida_leve_caminh_vigorosa'].isnull()) & (df['F30_minutos_corrida_leve_caminh_vigorosa'].isnull())
df.loc[condition, ['F29_dias_corrida_leve_caminh_vigorosa', 'F30_minutos_corrida_leve_caminh_vigorosa']] = 0

In [ ]:
condition = (df['F32_corrida_vigorosa'] == 0) & (df['F33_dias_corrida_vigorosa'].isnull()) & (df['F34_minutos_corrida_vigorosa'].isnull())
df.loc[condition, ['F33_dias_corrida_vigorosa', 'F34_minutos_corrida_vigorosa']] = 0

In [ ]:
condition = (df['F36_musculacao'] == 0) & (df['F37_dias_musculacao'].isnull()) & (df['F38_minutos_musculacao'].isnull())
df.loc[condition, ['F37_dias_musculacao', 'F38_minutos_musculacao']] = 0

In [ ]:
condition = (df['F40_natacao_academia_campo_aberto'] == 0) & (df['F41_dias_natacao_academia_campo_aberto'].isnull()) & (df['F42_minutos_natacao_academia_campo_aberto'].isnull())
df.loc[condition, ['F41_dias_natacao_academia_campo_aberto', 'F42_minutos_natacao_academia_campo_aberto']] = 0

In [ ]:
condition = (df['F44_voleibol'] == 0) & (df['F45_dias_voleibol'].isnull()) & (df['F46_minutos_voleibol'].isnull())
df.loc[condition, ['F45_dias_voleibol', 'F46_minutos_voleibol']] = 0

In [ ]:
threshold = 0.75
empty_cols = [col for col in df.columns if df[col].isnull().sum() / len(df) > threshold]
print(f"{len(empty_cols)}")
print(f"{empty_cols}")

3
['J45_manter_se_atualizado', 'J46_capaz_prestar_atencao_entender_programas', 'R4_import_relig_espiritual_vida']


Todo: Drop irelevant columns


In [ ]:
df.drop(columns=['N_protocolo'], inplace=True)


In [ ]:
df.drop(columns=['filter_$'], inplace=True)

Focar no texto;
desenvolver sessão objetivo na introdução
metodologia:como

In [ ]:
threshold = 0.5
empty_cols = [col for col in df.columns if df[col].isnull().sum() / len(df) > threshold]
print(f"{len(empty_cols)}")
print(f"{empty_cols}")

122
['C41_memoria', 'C42_orientacao_temporal_espacial', 'C43_julgamento_solucao_problemas', 'C44_relacoes_comunitarias', 'C45_lar_passa_tempos', 'C46_cuidados_pessoais', 'C47_CDR_soma_pontos', 'C49_CDR_nivel_demencia_Morris', 'F2_quantos_quilos', 'F4_caminhada', 'F5_dias_caminhada', 'F6_minutos_caminhada', 'F7_caminhada_3.8_METS', 'F8_ciclismo', 'F9_dias_ciclismo', 'F10_minutos_ciclismo', 'F11_ciclismo_4.0_METS', 'F12_danca_salao', 'F13_dias_danca_salao', 'F14_minutos_danca_salao', 'F15_danca_salao_4.5_METS', 'F16_ginastica_casa', 'F17_dias_ginastica_casa', 'F18_minutos_ginastica_casa', 'F19_ginastica_casa_3.5_METS', 'F20_ginastica_fora_casa', 'F21_dias_ginastica_fora_casa', 'F22_minutos_ginastica_fora_casa', 'F23_ginastica_fora_casa_6.0_METS', 'F24_hidroginastica', 'F25_dias_hidroginastica', 'F26_minutos_hidroginastica', 'F27_hidroginastica_4.0_METS', 'F28_corrida_leve_caminh_vigorosa', 'F29_dias_corrida_leve_caminh_vigorosa', 'F30_minutos_corrida_leve_caminh_vigorosa', 'F31_corrida_l

In [ ]:

for col in ['F8_ciclismo', 'F9_dias_ciclismo', 'F10_minutos_ciclismo', 'F11_ciclismo_4.0_METS', 'F12_danca_salao', 'F13_dias_danca_salao', 'F14_minutos_danca_salao', 'F15_danca_salao_4.5_METS', 'F16_ginastica_casa', 'F17_dias_ginastica_casa', 'F18_minutos_ginastica_casa', 'F19_ginastica_casa_3.5_METS', 'F20_ginastica_fora_casa', 'F21_dias_ginastica_fora_casa', 'F22_minutos_ginastica_fora_casa', 'F23_ginastica_fora_casa_6.0_METS']:
  df[col].fillna(0, inplace=False)
